# BBLF AI Selector: Part 1: Pre Tournament Optimal Squad - Optimser

# 0. Prerequistes

In [1]:
# 0. Prerequistes

import pandas as pd
import numpy as np
import os
import random
from mip import Model, xsum, maximize, BINARY 

os.getcwd()
directory = 'C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI'

# 1. Data Extraction 

In [2]:
# 1. Data Extraction 
    # Optimal Number of Rounds
opt_round = 3

    # Pull in player price data csv file 

price_df = pd.read_csv(os.path.join(directory,'data/python_datasets/player_price_round1_fix.csv'), low_memory=False)
print(price_df)

# Player Role Flags

price_df["Wk_f"] = np.where((price_df["Role"] == "WK"), 1, 0)
price_df["Bat_f"] = np.where((price_df["Role"] == "WK") |(price_df["Role"] == "BAT") | (price_df["Role"] == "ALLR") , 1, 0)
price_df["Bowl_f"] = np.where((price_df["Role"] == "BOWL") | (price_df["Role"] == "ALLR") , 1, 0)
price_df = price_df[["Full_Name", "Price", "Team","Wk_f", "Bat_f", "Bowl_f", "Role", "Available"]].rename(columns = {"Full_Name": "Name"}) 
    # Pull in team fixture csv file

#team_fix_df = pd.read_csv(os.path.join(directory,'data/python_datasets/team_fixture.csv'), low_memory=False)
#team_fix_df = team_fix_df.melt(id_vars = "Team")
#team_fix_df = team_fix_df.rename(columns = {"variable": "Round", "value": "Opposition"})
#team_fix_df[["Round", "Game"]] = team_fix_df["Round"].str.split("_", expand = True)
#team_fix_df["Round"] = team_fix_df["Round"].str[1:].astype(int)
#team_fix_df = team_fix_df[["Team", "Round", "Opposition"]]
#team_fix_df = team_fix_df[team_fix_df.Round <= opt_round].dropna()
#print(team_fix_df)

team_fix_df = pd.read_csv(os.path.join(directory,'data/python_datasets/team_loc_fixture.csv'), low_memory=False)
team_fix_df = team_fix_df[team_fix_df.Round <= opt_round].dropna()
print(team_fix_df)

    # Pull player expected points csv file
exp_pts_df = pd.read_csv(os.path.join(directory,'data/python_datasets/bbl14_fullteam_model_score.csv'), low_memory=False).drop(["Unnamed: 0", "player"], axis = 1)
print(exp_pts_df)

    # Join team fixture to player price to create a row for each game
game_df = pd.merge(price_df , team_fix_df, left_on = ["Team"], right_on = ["Team"], how = "left")
print(game_df)

    # Join on expected points for each game
print(game_df.columns)
print(exp_pts_df.columns)
player_df_raw = pd.merge(game_df, exp_pts_df, left_on = ["Name", "Team", "Opposition", "Venue", "Home_f"], right_on = ["Name", "Team", "opp", "venue", "Home_f"], how = "left")
#player_df_raw["exp_pts"] = np.random.randint(0, 125, player_df_raw.shape[0])
player_df_raw["weight"] = 1
print(player_df_raw)

    # Aggregate by player name (calculate total expected points for each player for x number of rounds)
player_df = player_df_raw.groupby(['Name', 'Price', "Team", "Wk_f", "Bat_f", "Bowl_f", "Role", "weight","Available"], as_index=False).agg(
exp_pts=('exp_pts',"sum"))

      Last_Name First_Name First_Name_Short        Full_Name         player  \
0        Abbott       Sean               SA      Sean Abbott      SA Abbott   
1          Agar     Ashton               AC      Ashton Agar        AC Agar   
2          Agar        Wes               WA         Wes Agar        WA Agar   
3         Allen       Finn              NaN       Finn Allen     Finn Allen   
4         Allen     Fabian              NaN     Fabian Allen   Fabian Allen   
..          ...        ...              ...              ...            ...   
142       Wells        Jon               JW        Jon Wells       JW Wells   
143  Wildermuth       Jack               JD  Jack Wildermuth  JD Wildermuth   
144        Wood       Jack               JP        Jack Wood        JP Wood   
145      Wright        Mac                M       Mac Wright       M Wright   
146       Zampa       Adam                A       Adam Zampa        A Zampa   

      Price                 Team  Role  Available  

# 2. Optimisation Process

Optimisation Objective: Maximise the number of expected fantasy points

Constraints: 
1. Number of players selected must be 4
2. Atleast NA wicketkeeper
3. Atleast 2 batters
4. Atleast 1 bowlers
5. Total budget of team is less than $887000

In [3]:
# Optimisation Setup

points = player_df["exp_pts"]

price = player_df["Price"]

#model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"] = model_df.loc[:,"opp_Adelaide Strikers":"venue_WACA"].astype(object)

weight = player_df["weight"]
available = player_df["Available"]
wk_weight = player_df["Wk_f"]
bat_weight = player_df["Bat_f"]
bowl_weight = player_df["Bowl_f"]

play_cnt, total_player = 4, range(len(price))
wk_cnt, total_wk = 0, range(len(price))
bat_cnt, total_bat = 2, range(len(price))
bowl_cnt, total_bowl = 1, range(len(price))
budget, total_budget = 887000, range(len(price))

m = Model("knapsack")

x = [m.add_var(var_type=BINARY) for i in total_player]
print(x)

m.objective = maximize(xsum(points[i]*x[i] for i in total_player))

m += xsum(weight[i] * x[i] for i in total_player) == play_cnt
m += xsum(wk_weight[i] * x[i] for i in total_wk) >= wk_cnt
m += xsum(bat_weight[i] * x[i] for i in total_bat) >= bat_cnt
m += xsum(bowl_weight[i] * x[i] for i in total_bowl) >= bowl_cnt
m += xsum(price[i] * x[i] for i in total_budget) <= budget
m += xsum(available[i] * x[i] for i in total_player) == play_cnt

m.optimize()

selected = [i for i in total_player if x[i].x >= 0.99]
print("selected items: {}".format(selected))

sel_player_df = player_df.iloc[selected]

print("Total Expect Points:", sum(sel_player_df["exp_pts"]))
print("Total Team Cost:", sum(sel_player_df["Price"]))
print("Number of Wk:", sum(sel_player_df["Wk_f"]))
print("Number of Bat:", sum(sel_player_df["Bat_f"]))
print("Number of Bowl:", sum(sel_player_df["Bowl_f"]))
print("Available Players:", sum(sel_player_df["Available"]))

print(sel_player_df.sort_values(by = "exp_pts", ascending = False))

sel_player_df.to_csv('C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI/data/python_datasets/optimal_4_remaining_post_mistake.csv')


[<mip.entities.Var object at 0x0000018C8D03C250>, <mip.entities.Var object at 0x0000018C8D03C2B0>, <mip.entities.Var object at 0x0000018C8D03C6D0>, <mip.entities.Var object at 0x0000018C8D03C7C0>, <mip.entities.Var object at 0x0000018C8D03C820>, <mip.entities.Var object at 0x0000018C8D03C880>, <mip.entities.Var object at 0x0000018C8D03C6A0>, <mip.entities.Var object at 0x0000018C8D03C730>, <mip.entities.Var object at 0x0000018C8D03C7F0>, <mip.entities.Var object at 0x0000018C8D03C790>, <mip.entities.Var object at 0x0000018C8D03C910>, <mip.entities.Var object at 0x0000018C8D03C4F0>, <mip.entities.Var object at 0x0000018C8D03C5B0>, <mip.entities.Var object at 0x0000018C8D03C9A0>, <mip.entities.Var object at 0x0000018C8D03C8E0>, <mip.entities.Var object at 0x0000018C8D03C940>, <mip.entities.Var object at 0x0000018C8D03CA30>, <mip.entities.Var object at 0x0000018C8D03C9D0>, <mip.entities.Var object at 0x0000018C8D03CA00>, <mip.entities.Var object at 0x0000018C8D03C850>, <mip.entities.Var o

# 1b. Data Extraction (Best Bench Trick Player)

In [3]:
# 1. Data Extraction 
    # Optimal Number of Rounds
opt_round = 2

    # Pull in player price data csv file 

price_df = pd.read_csv(os.path.join(directory,'data/python_datasets/player_price_round1_fix_roll_window.csv'), low_memory=False)
print(price_df)

# Player Role Flags

price_df["Wk_f"] = np.where((price_df["Role"] == "WK"), 1, 0)
price_df["Bat_f"] = np.where((price_df["Role"] == "WK") |(price_df["Role"] == "BAT") | (price_df["Role"] == "ALLR") , 1, 0)
price_df["Bowl_f"] = np.where((price_df["Role"] == "BOWL") | (price_df["Role"] == "ALLR") , 1, 0)
price_df = price_df[["Full_Name", "Price", "Team","Wk_f", "Bat_f", "Bowl_f", "Role", "Available"]].rename(columns = {"Full_Name": "Name"}) 
    # Pull in team fixture csv file

#team_fix_df = pd.read_csv(os.path.join(directory,'data/python_datasets/team_fixture.csv'), low_memory=False)
#team_fix_df = team_fix_df.melt(id_vars = "Team")
#team_fix_df = team_fix_df.rename(columns = {"variable": "Round", "value": "Opposition"})
#team_fix_df[["Round", "Game"]] = team_fix_df["Round"].str.split("_", expand = True)
#team_fix_df["Round"] = team_fix_df["Round"].str[1:].astype(int)
#team_fix_df = team_fix_df[["Team", "Round", "Opposition"]]
#team_fix_df = team_fix_df[team_fix_df.Round <= opt_round].dropna()
#print(team_fix_df)

team_fix_df = pd.read_csv(os.path.join(directory,'data/python_datasets/team_loc_fixture.csv'), low_memory=False)
team_fix_df = team_fix_df[team_fix_df.Round <= opt_round].dropna()
print(team_fix_df)

    # Pull player expected points csv file
exp_pts_df = pd.read_csv(os.path.join(directory,'data/python_datasets/bbl14_fullteam_model_score.csv'), low_memory=False).drop(["Unnamed: 0", "player"], axis = 1)
print(exp_pts_df)

    # Join team fixture to player price to create a row for each game
game_df = pd.merge(price_df , team_fix_df, left_on = ["Team"], right_on = ["Team"], how = "left")
print(game_df)

    # Join on expected points for each game
print(game_df.columns)
print(exp_pts_df.columns)
player_df_raw = pd.merge(game_df, exp_pts_df, left_on = ["Name", "Team", "Opposition", "Venue", "Home_f"], right_on = ["Name", "Team", "opp", "venue", "Home_f"], how = "left")
#player_df_raw["exp_pts"] = np.random.randint(0, 125, player_df_raw.shape[0])
player_df_raw["weight"] = 1
print(player_df_raw)

    # Aggregate by player name (calculate total expected points for each player for x number of rounds)
player_df = player_df_raw.groupby(['Name', 'Price', "Team", "Wk_f", "Bat_f", "Bowl_f", "Role", "weight","Available"], as_index=False).agg(
exp_pts=('exp_pts',"sum"))

      Last_Name First_Name First_Name_Short        Full_Name         player  \
0        Abbott       Sean               SA      Sean Abbott      SA Abbott   
1          Agar     Ashton               AC      Ashton Agar        AC Agar   
2          Agar        Wes               WA         Wes Agar        WA Agar   
3         Allen       Finn              NaN       Finn Allen     Finn Allen   
4         Allen     Fabian              NaN     Fabian Allen   Fabian Allen   
..          ...        ...              ...              ...            ...   
142       Wells        Jon               JW        Jon Wells       JW Wells   
143  Wildermuth       Jack               JD  Jack Wildermuth  JD Wildermuth   
144        Wood       Jack               JP        Jack Wood        JP Wood   
145      Wright        Mac                M       Mac Wright       M Wright   
146       Zampa       Adam                A       Adam Zampa        A Zampa   

      Price                 Team  Role  Available  

# 2b. Optimisation Process

Optimisation Objective: Maximise the number of expected fantasy points

Constraints: 
1. Number of players selected must be 1
2. Atleast NA wicketkeeper
3. Atleast NA batters
4. Atleast NA bowlers
5. Total budget of team is less than $155900

In [4]:
# Optimisation Setup

points = player_df["exp_pts"]

price = player_df["Price"]

weight = player_df["weight"]
available = player_df["Available"]
wk_weight = player_df["Wk_f"]
bat_weight = player_df["Bat_f"]
bowl_weight = player_df["Bowl_f"]

play_cnt, total_player = 1, range(len(price))
wk_cnt, total_wk = 0, range(len(price))
bat_cnt, total_bat = 0, range(len(price))
bowl_cnt, total_bowl = 0, range(len(price))
budget, total_budget = 155900, range(len(price))

m = Model("knapsack")

x = [m.add_var(var_type=BINARY) for i in total_player]
print(x)

m.objective = maximize(xsum(points[i]*x[i] for i in total_player))

m += xsum(weight[i] * x[i] for i in total_player) == play_cnt
m += xsum(wk_weight[i] * x[i] for i in total_wk) >= wk_cnt
m += xsum(bat_weight[i] * x[i] for i in total_bat) >= bat_cnt
m += xsum(bowl_weight[i] * x[i] for i in total_bowl) >= bowl_cnt
m += xsum(price[i] * x[i] for i in total_budget) <= budget
m += xsum(available[i] * x[i] for i in total_player) == play_cnt

m.optimize()

selected = [i for i in total_player if x[i].x >= 0.99]
print("selected items: {}".format(selected))

sel_player_df = player_df.iloc[selected]

print("Total Expect Points:", sum(sel_player_df["exp_pts"]))
print("Total Team Cost:", sum(sel_player_df["Price"]))
print("Number of Wk:", sum(sel_player_df["Wk_f"]))
print("Number of Bat:", sum(sel_player_df["Bat_f"]))
print("Number of Bowl:", sum(sel_player_df["Bowl_f"]))
print("Available Players:", sum(sel_player_df["Available"]))

print(sel_player_df.sort_values(by = "exp_pts", ascending = False))

sel_player_df.to_csv('C:/Users/dilan/OneDrive/Documents/Data Science Projects/Big Bash Fantasy AI/data/python_datasets/bench_trick_remaining_post_mistake.csv')


[<mip.entities.Var object at 0x0000020EF47245B0>, <mip.entities.Var object at 0x0000020EF4724730>, <mip.entities.Var object at 0x0000020EF4724820>, <mip.entities.Var object at 0x0000020EF47248B0>, <mip.entities.Var object at 0x0000020EF47247C0>, <mip.entities.Var object at 0x0000020EF47248E0>, <mip.entities.Var object at 0x0000020EF4724760>, <mip.entities.Var object at 0x0000020EF4724940>, <mip.entities.Var object at 0x0000020EF47244C0>, <mip.entities.Var object at 0x0000020EF4724970>, <mip.entities.Var object at 0x0000020EF4724A00>, <mip.entities.Var object at 0x0000020EF47246A0>, <mip.entities.Var object at 0x0000020EF4724610>, <mip.entities.Var object at 0x0000020EF4724880>, <mip.entities.Var object at 0x0000020EF47249D0>, <mip.entities.Var object at 0x0000020EF47249A0>, <mip.entities.Var object at 0x0000020EF4724A60>, <mip.entities.Var object at 0x0000020EF4724AC0>, <mip.entities.Var object at 0x0000020EF4724850>, <mip.entities.Var object at 0x0000020EF4724B20>, <mip.entities.Var o